# **Objetivo del Cuaderno Jupyter**
- Procesar el archivo `movies_dataset.csv` y dejarlo de una manera consumible en `movies.parquet`, aunque todavía falte desanidar algunos campos internos (que se desanidaran en el próximo Cuaderno Jupyter), se consigue obtener una data mas `Limpia y Precisa` para su posterior análisis.
- Cumplir con los `Requerimientos de Aprobación` en el apartado de `Transformaciones` de este proyecto
- Preparar la `data a la medida` de las `6 primeras funciones de la Api`
    - cantidad_filmaciones_mes:
        - api_01_cantidad_filmaciones_mes.parquet
    - cantidad_filmaciones_dia:
        - api_02_cantidad_filmaciones_dia.parquet
    - score_titulo:
        - api_03_04_titulos.parquet
    - votos_titulo:
        - api_03_04_titulos.parquet
    - get_actor:
        - api_05_get_actor.parquet
    - get_director:
        - api_06_get_director.parquet
        - directores.parquet

In [1]:
import polars as pl

### Nombres de los campos presentes en el archivo `movies_dataset.csv` <br>
<img src='assets/diccionario_datos.png'>

### Funcion que recibe una Linea del archivo `movies_dataset.csv` y devuelve todos los campos en un diccionario `fields` <br>

**Nota** Tomé la decisión de Extraer los Datos de esta forma, ya que al cargarlo con Pandas, me alerto un Warning donde me indicaba la inferencia de tipos mixtos de datos.  Al utilizar la libreria Polars nisiquiera pudo abrirlo, lo que me llevó a la precaución de explorar los datasets originales de una forma mas básica. Básicamente la Librería Pandas hubiese sido suficiente para continuar, pero quise ir sobre seguro en la precisión de la data obtenida.<br>

Para entender el codigo se hace necesario leer el Docstring, Tomar en cuenta que MINAR de Comentarios el Codigo lo va a hacer mas confuso. Preferí documentar bien el docstring, luego abajo doy un ejemplo de como devuelve la data esta función (ayuda a entender que hace el codigo) y lo mas importante para entender el codigo es prestar MUCHA ATENCION al NOMBRE DE LAS VARIABLES de tipo BOLLEANO, ya que esto ayuda mucho a la hora de leer los condicionales de la función. 

# **A Pie**

In [2]:
def get_fields_from_line(line:str)-> dict:
    """get_fields_from_line: 
           Función que recibe una linea del dataset `movies_dataset.csv` y devuelve una variable tipo
        diccionario contentiva de la informacion de los campos.
        EL tipo de datos que tiene el dataset es el que se muestra en la imagen arriba.
    
    Puntos a tomar en cuenta:
        .- La variable `fields_names` es una Lista que contiene el nombre de los campos en el orden en que aparecen 
           en el Dataset, lo que ayuda al algoritmo que se encargará de extraer los campos de cada Linea del archivo 
           'csv', el cual se guiará por aca para la asignación de los nombres de los keys del diccionario que devuelve
        .- La variable `index_fields` lleva la cuenta de cuantos campos han ingresado al diccionario, lo que sirve 
           como guia para indexar dentro de la variable `fields_names`. Tambien sirve para añadir el valor que se le 
           da a el campo `Numero_campos`
        .- La variable `index` es la usada en el ciclo for para recorrer la linea caracter a caracter
        .- La variable `previous_index` es el indice de la posicion donde inicia cada slice `[:]` que extrae un campo 
           en específico.
        .- En el dataset hay tipos de datos como Diccionarios, Listas y String Largos que vienen entre comillas `"`, 
           para lo cual utilizo la variable `entre_comillas` para indicar en el codigo que el `index` se encuentra en 
           alguna posicion entre las comillas `"` que abren y las que cierran.       

    Nota: Los nombres de las Variables y la manera como se estructuraron las validaciones hace que el codigo cuente un poco lo que hace
    Ejemplo: en la validación  if not entre_comillas and line[index] == ',':    
             lo quiere indicar, es que el indice `index` está en una parte de la linea NO esta encerrda entre comillas y que
             el caracter que tiene la linea en el indice `index` es igual a como `,`   
    Args:
        line (str): Linea del dataset `movies_dataset`

    Returns:
        dict: Diccionario con todos los campos encontrados en la linea recibida
    """
    fields_names:list = ["adult", "belongs_to_collection", "budget", "genres", "homepage", "id", "imdb_id",
                         "original_language", "original_title", "overview", "popularity", "poster_path",
                         "production_companies", "production_countries", "release_date", "revenue", "runtime",
                         "spoken_languages", "status", "tagline", "title", "video", "vote_average", "vote_count"]
    index_fields:int = 0
    previous_index:int = 0
    entre_comillas:bool = False
    fields:dict = {}
    for index in range(len(line)):
        if index == len(line)-1:
            cadena = line[previous_index:]
            fields[fields_names[index_fields]] = cadena
            fields['Numero_Campos'] = index_fields + 1
            break
        if not entre_comillas and line[index] == ',':
            cadena = line[previous_index:index]
            fields[fields_names[index_fields]] = cadena
            index_fields += 1
            previous_index = index+1
        elif not entre_comillas and line[index] == '"':
            entre_comillas = True
            previous_index = index+1
        elif entre_comillas and index > previous_index:
            if line[index] == ',' and line[index - 1] == '"' and line[index - 2] != '"':      
                cadena = line[previous_index:index-1]
                fields[fields_names[index_fields]] = cadena
                index_fields += 1
                previous_index = index + 1
                entre_comillas = False

    return fields

Abro el archivo movies_dataset.csv como texto plano lo recorro y cada linea la mando a desglozar con la función `get_fields_from_line` y los diccionarios recibidos los voy guardando en una lista en una variable llamada `data`

# **POR FAVOR LEER**
### **POR FAVOR LEA ESTE MENSAJE ANTES DE CONTINUAR**

Para poder continuar Necesitamos el archivo `movies_dataset.csv` que nos dieron al empezar el Proyecto:<br>

**OPCIONES**
- Sí dispone del archivo `movies_dataset.csv` coloquelo en la carpeta `datasets_inicial` que está este proyecto y `NO EJECUTE LA CELDA 2` Osea la siguiente celda.
- Sí por el contrario NO DISPONE del archivo, entonces ejecute la siguiente celda para descargarlo.

In [1]:
import gdown

url = 'https://drive.google.com/uc?export=download&id=1kCjJAxp9Gnawhbvv_VgkQBQrQ6PJd906'

gdown.download(url, 'datasets_inicial/movies_dataset.csv', quiet=False)

Downloading...
From: https://drive.google.com/uc?export=download&id=1kCjJAxp9Gnawhbvv_VgkQBQrQ6PJd906
To: d:\BOOTCAMP-HENRY\MODULO-07-LABS\datasets_inicial\movies_dataset.csv
100%|██████████| 34.4M/34.4M [00:12<00:00, 2.69MB/s]


'datasets_inicial/movies_dataset.csv'

Recorro el archivo como si fuera texto y cada linea lo mando a la función

In [3]:
data = []
with open("datasets_inicial/movies_dataset.csv", 'r', encoding='utf-8') as file:
    next(file)
    for line in file:
        line = line.strip()
        row = get_fields_from_line(line=line) #Mando cada linea a la funcion de arriba
        data.append(row)
    file.close()


### Ejemplo de como viene la Data proveniente de `get_fields_from_line`

In [4]:
# Aca se muestra una Linea del archivo `movies_dataset.csv`
print(f'En la variable data hay un total de {len(data)} registros ...')
data[1001]

En la variable data hay un total de 45572 registros ...


{'adult': 'False',
 'belongs_to_collection': '',
 'budget': '812000',
 'genres': "[{'id': 16, 'name': 'Animation'}, {'id': 10751, 'name': 'Family'}]",
 'homepage': '',
 'id': '11360',
 'imdb_id': 'tt0033563',
 'original_language': 'en',
 'original_title': 'Dumbo',
 'overview': 'Dumbo is a baby elephant born with oversized ears and a supreme lack of confidence. But thanks to his even more diminutive buddy -- Timothy the Mouse -- the pint-sized pachyderm learns to surmount all obstacles.',
 'popularity': '14.655879',
 'poster_path': '/r5IqhwZ61OuKlsyDwvXWyWQZK30.jpg',
 'production_companies': "[{'name': 'RKO Radio Pictures', 'id': 6}, {'name': 'Walt Disney Productions', 'id': 3166}]",
 'production_countries': "[{'iso_3166_1': 'US', 'name': 'United States of America'}]",
 'release_date': '1941-10-22',
 'revenue': '1600000',
 'runtime': '64.0',
 'spoken_languages': "[{'iso_639_1': 'en', 'name': 'English'}]",
 'status': 'Released',
 'tagline': 'The One...The Only...The FABULOUS...',
 'title

Observo la data ya transformada a Dataframe

In [6]:
movies = pl.DataFrame(data=data)
print(movies.shape)
movies.head(1)

(45572, 25)


adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,Numero_Campos
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64
"""False""","""{'id': 10194, 'name': 'Toy Sto…","""30000000""","""[{'id': 16, 'name': 'Animation…","""http://toystory.disney.com/toy…","""862""","""tt0114709""","""en""","""Toy Story""","""Led by Woody, Andy's toys live…","""21.946943""","""/rhIRbceoE9lR4veEXuwCC2wARtG.j…","""[{'name': 'Pixar Animation Stu…","""[{'iso_3166_1': 'US', 'name': …","""1995-10-30""","""373554033""","""81.0""","""[{'iso_639_1': 'en', 'name': '…","""Released""","""""","""Toy Story""","""False""","""7.7""","""5415""",24


La Funcion `get_fields_from_line` además de obtener los datos de las columnas tambien agregó un campo a cada registro que indica el número de campos que se obtuvieron en el mismo. Al haber sido yo quien programó la función que extrae los datos, tengo seguridad de la calidad de los datos (pero solo de las lineas con 24 campos) La data venía con multiples Lineas incompletas, no solo de campos vacios, sino que tambien registros sin todos los campos. Por observacion algo mas detallada tambien vi que los registros que venían sin sus 24 campos a veces eran incluso una sola cadena de caracteres y en otros casos datos malos, que no merecen ser tomados en cuenta. Lo ideal, `en este caso específico` (y cada caso es distinto y hay que estudiarlo por separado), es que cada registro tenga 24 campos, ya que es la GRAN MAYORIA de los registros.


In [7]:
mask = movies['Numero_Campos'] == 24
registros_totales = movies.shape[0]
registros_con_24_campos = movies.filter(mask).shape[0]
porcentaje_registros_incompletos =  round(100 * ((registros_totales - registros_con_24_campos) / registros_totales),2)

print(f'Cantidad de Registros con sus 24 campos --> {registros_con_24_campos}')
print(f'Porcentaje de Registros que NO TIENEN 24 campos --> {porcentaje_registros_incompletos} %')
movies['Numero_Campos'].value_counts()

Cantidad de Registros con sus 24 campos --> 45010
Porcentaje de Registros que NO TIENEN 24 campos --> 1.23 %


Numero_Campos,count
i64,u32
22,1
4,9
11,1
20,148
13,12
…,…
24,45010
14,12
21,176


Elimino las lineas que no tienen 24 campos

In [8]:
mask = movies['Numero_Campos'] == 24
movies = movies.filter(mask)

La linea 10145 del dataset `movies_dataset.csv`, logró burlar el modesto algoritmo realizado para la extracción de los campos, al ser una linea con un solo texto, sin comillas dobles `"` y con justo 23 comas `,`  <br>
**Nota**: Pudiera dedicarle unas Horas (que pueden transformarse en un dia o algo mas) en perfeccionar el Algoritmo, pero basado en la Premisa `MVP` exigida en el archivo `README` de este proyecto, tambien en que fue una sola linea la que logró colarse, y sobretodo, en que esa linea es data basura, eliminaré el registro a PIE para ganar tiempo 

In [9]:
# Eliminando Fila de Malos Datos que se le coló al Algoritmo
mask = movies['popularity'] != " whose cynicism has been lifted by the boy's pure hope"
movies = movies.filter(mask)

***
# **Requerimientos ETL Exigidos en el Readme**

### **Requerimiento**
+ Eliminar las columnas que no serán utilizadas, **`video`**,**`imdb_id`**,**`adult`**,**`original_title`**,**`poster_path`** y **`homepage`**. <br>
**Nota**: Se Eliminará también la columna `Numero_Campos`, ya que su único propósito era identificar las filas que no tenían los 24 campos, para Eliminarlas (y ya el objetivo fue cumplido)

In [10]:
movies = movies.drop(['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage', 'Numero_Campos'])

***
### **Requerimiento**
+ Los valores nulos de los campos **`revenue`**, **`budget`** deben ser rellenados por el número **`0`**.

Nota: La función `get_fields_from_line` **NO RETORNO Valores Nulos, ni NAN**. Lo que devolvía en el `campo` cuando lo encontraba vacio fue `''` (Es decir: en este caso Nulo = "") Abajo se verá

In [11]:
for column in movies.columns:
    mask = movies[column] == ""
    print(f'Cantidad de Nulos --> {movies[column].is_null().sum()}   Columna --> {column}') # Sintaxis Normal para ver Nulos
    print(f'Cantidad de ""--> {movies.filter(mask).shape[0]}') 
    print('*'*50)

Cantidad de Nulos --> 0   Columna --> belongs_to_collection
Cantidad de ""--> 40553
**************************************************
Cantidad de Nulos --> 0   Columna --> budget
Cantidad de ""--> 0
**************************************************
Cantidad de Nulos --> 0   Columna --> genres
Cantidad de ""--> 0
**************************************************
Cantidad de Nulos --> 0   Columna --> id
Cantidad de ""--> 0
**************************************************
Cantidad de Nulos --> 0   Columna --> original_language
Cantidad de ""--> 11
**************************************************
Cantidad de Nulos --> 0   Columna --> overview
Cantidad de ""--> 953
**************************************************
Cantidad de Nulos --> 0   Columna --> popularity
Cantidad de ""--> 0
**************************************************
Cantidad de Nulos --> 0   Columna --> production_companies
Cantidad de ""--> 0
**************************************************
Cantidad de Nulos --> 0

**Nota**: Para asignar 0 a un campo este debe ser númerico... Como Todavía NO he casteado las columnas a sus respectivos formatos aprovecharé de hacer ese paso primero 

In [12]:
# Como el único Campo Numerico que presenta valores '' (osea Nulos) 
# es runtime pasaré a '0' esos registros para poder castear luego
movies = movies.with_columns(
    pl.when(pl.col('runtime') == '').then(pl.lit('0')).otherwise(pl.col('runtime')).alias('runtime')
)

# CASTEANDO los campos Númericos
movies = movies.with_columns([
    pl.col('budget').cast(pl.Float32).alias('budget'),
    pl.col('id').cast(pl.Int32).alias('id'),
    pl.col('popularity').cast(pl.Float32).alias('popularity'),
    pl.col('revenue').cast(pl.Float32).alias('revenue'),
    pl.col('runtime').cast(pl.Float32).alias('runtime'),
    pl.col('vote_average').cast(pl.Float32).alias('vote_average'),
    pl.col('vote_count').cast(pl.Int32).alias('vote_count'),   
])

Con la Data extraída de esta manera `revenue` y `budget` no tienen valores nulos, pero al tratarse esto de un ETL con fines educativos, la Sintáxis para rellenar de ceros(0), los respectivos valores Nulos, de las columnas nombradas, se muestra en la siguiente celda<br>

In [13]:
columns = ['revenue', 'budget']
for column in columns:
    movies = movies.with_columns(pl.col(column).fill_null(0))

***
### **Requerimiento**
+ Los valores nulos del campo **`release date`** deben eliminarse. <br>
Nota: En este caso NULO = ""

In [14]:
# Eliminando los registros con release_date = Nulo (osea = "")
mask = movies['release_date'] != ""
movies = movies.filter(mask)

# Verificando que no queden Nulos
mask = (movies['release_date'].is_null()) | (movies['release_date'] == "")
print(f'Cantidad de Nulos ó "" --> {movies.filter(mask).shape[0]}')

Cantidad de Nulos ó "" --> 0


***
### **Requerimiento**
+ Crear la columna con el retorno de inversión, llamada **`return`** con los campos **`revenue`** y **`budget`**, dividiendo estas dos últimas **`revenue / budget`**, cuando no hay datos disponibles para calcularlo, deberá tomar el valor **`0`**.

In [15]:
# Nota: El denominador (en este caso `budget`) es el único que debe ser distinto de 0 para poder aplicar la formula
# Cuando budget es distinto de 0 entonces `revenue`/`budget` en cualquier otro caso 0
movies = movies.with_columns(
    pl.when(pl.col('budget') != 0)
      .then(pl.col('revenue') / pl.col('budget'))
      .otherwise(pl.lit(0))
      .alias('return')
)

In [16]:
movies.head(1)

belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,return
str,f32,str,i32,str,str,f32,str,str,str,f32,f32,str,str,str,str,f32,i32,f32
"""{'id': 10194, 'name': 'Toy Sto…",3e7,"""[{'id': 16, 'name': 'Animation…",862,"""en""","""Led by Woody, Andy's toys live…",21.946943,"""[{'name': 'Pixar Animation Stu…","""[{'iso_3166_1': 'US', 'name': …","""1995-10-30""",3.73554048e8,81.0,"""[{'iso_639_1': 'en', 'name': '…","""Released""","""""","""Toy Story""",7.7,5415,12.451801


***
### **Requerimiento**
+ De haber fechas, deberán tener el formato **`AAAA-mm-dd`**, además deberán crear la columna **`release_year`** donde extraerán el año de la fecha de estreno.

In [17]:
# Casteando la Columna a Date
movies = movies.with_columns(
    pl.col('release_date').cast(pl.Date).alias('release_date')
) 

Creando columna `release_year` <br>
**Nota**: Aprovecharé de crear la columna `release_month` que tendrá el numero del mes del estreno y la columna `release_day_of_week` que tendrá el número del dia de la semana del estreno


In [18]:
movies = movies.with_columns([
        pl.col("release_date").dt.year().alias("release_year"), # Año
        pl.col("release_date").dt.month().alias("release_month"), # Número del mes
        pl.col("release_date").dt.weekday().alias("release_day_of_week"), # Número de dia en la semana
    ])

***
- Necesito crear una Columna con el `Nombre del Mes en Español` de los Estrenos y otra con el `Nombre del dia en Español`

In [19]:
movies = movies.with_columns(
    name_of_month = pl.when(pl.col('release_month') == 1).then(pl.lit("ENERO"))
                      .when(pl.col('release_month') == 2).then(pl.lit("FEBRERO"))
                      .when(pl.col('release_month') == 3).then(pl.lit("MARZO"))
                      .when(pl.col('release_month') == 4).then(pl.lit("ABRIL"))
                      .when(pl.col('release_month') == 5).then(pl.lit("MAYO"))
                      .when(pl.col('release_month') == 6).then(pl.lit("JUNIO"))
                      .when(pl.col('release_month') == 7).then(pl.lit("JULIO"))
                      .when(pl.col('release_month') == 8).then(pl.lit("AGOSTO"))
                      .when(pl.col('release_month') == 9).then(pl.lit("SEPTIEMBRE"))
                      .when(pl.col('release_month') == 10).then(pl.lit("OCTUBRE"))
                      .when(pl.col('release_month') == 11).then(pl.lit("NOVIEMBRE"))
                      .otherwise(pl.lit('DICIEMBRE'))
)

In [20]:
movies = movies.with_columns(
    name_of_day = pl.when(pl.col('release_day_of_week') == 1).then(pl.lit("LUNES"))
                    .when(pl.col('release_day_of_week') == 2).then(pl.lit("MARTES"))
                    .when(pl.col('release_day_of_week') == 3).then(pl.lit("MIERCOLES"))
                    .when(pl.col('release_day_of_week') == 4).then(pl.lit("JUEVES"))
                    .when(pl.col('release_day_of_week') == 5).then(pl.lit("VIERNES"))
                    .when(pl.col('release_day_of_week') == 6).then(pl.lit("SABADO"))
                    .otherwise(pl.lit('DOMINGO'))
)

In [21]:
movies.head()

belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,return,release_year,release_month,release_day_of_week,name_of_month,name_of_day
str,f32,str,i32,str,str,f32,str,str,date,f32,f32,str,str,str,str,f32,i32,f32,i32,i8,i8,str,str
"""{'id': 10194, 'name': 'Toy Sto…",3e7,"""[{'id': 16, 'name': 'Animation…",862,"""en""","""Led by Woody, Andy's toys live…",21.946943,"""[{'name': 'Pixar Animation Stu…","""[{'iso_3166_1': 'US', 'name': …",1995-10-30,3.73554048e8,81.0,"""[{'iso_639_1': 'en', 'name': '…","""Released""","""""","""Toy Story""",7.7,5415,12.451801,1995,10,1,"""OCTUBRE""","""LUNES"""
"""""",6.5e7,"""[{'id': 12, 'name': 'Adventure…",8844,"""en""","""When siblings Judy and Peter d…",17.015539,"""[{'name': 'TriStar Pictures', …","""[{'iso_3166_1': 'US', 'name': …",1995-12-15,2.62797248e8,104.0,"""[{'iso_639_1': 'en', 'name': '…","""Released""","""Roll the dice and unleash the …","""Jumanji""",6.9,2413,4.043035,1995,12,5,"""DICIEMBRE""","""VIERNES"""
"""{'id': 119050, 'name': 'Grumpy…",0.0,"""[{'id': 10749, 'name': 'Romanc…",15602,"""en""","""A family wedding reignites the…",11.7129,"""[{'name': 'Warner Bros.', 'id'…","""[{'iso_3166_1': 'US', 'name': …",1995-12-22,0.0,101.0,"""[{'iso_639_1': 'en', 'name': '…","""Released""","""Still Yelling. Still Fighting.…","""Grumpier Old Men""",6.5,92,0.0,1995,12,5,"""DICIEMBRE""","""VIERNES"""
"""""",1.6e7,"""[{'id': 35, 'name': 'Comedy'},…",31357,"""en""","""Cheated on, mistreated and ste…",3.859495,"""[{'name': 'Twentieth Century F…","""[{'iso_3166_1': 'US', 'name': …",1995-12-22,8.145216e7,127.0,"""[{'iso_639_1': 'en', 'name': '…","""Released""","""Friends are the people who let…","""Waiting to Exhale""",6.1,34,5.09076,1995,12,5,"""DICIEMBRE""","""VIERNES"""
"""{'id': 96871, 'name': 'Father …",0.0,"""[{'id': 35, 'name': 'Comedy'}]""",11862,"""en""","""Just when George Banks has rec…",8.387519,"""[{'name': 'Sandollar Productio…","""[{'iso_3166_1': 'US', 'name': …",1995-02-10,7.6578912e7,106.0,"""[{'iso_639_1': 'en', 'name': '…","""Released""","""Just When His World Is Back To…","""Father of the Bride Part II""",5.7,173,0.0,1995,2,5,"""FEBRERO""","""VIERNES"""


***
# **Trabajando en Funciones de la API**


## **Trabajando en `cantidad_filmaciones_mes`**

**REQUERIMIENTO DE LA FUNCION**<br>
Se ingresa un `mes en idioma Español`. Debe devolver la `cantidad de películas` que fueron estrenadas en el mes consultado en la totalidad del dataset.

In [22]:
# Obtengo una lista con los nombres de los meses 
months = movies['name_of_month'].unique().to_list()

# Cada fila se mete (en cada iteracion) en un diccionario llamado `row` y estos a su vez en una lista llamada `data`
data = []
for month in months:
    row = {}
    # Filtrando el diccionario con el mes de la iteracion en curso
    mask = movies['name_of_month'] == month
    row['month'] = month
    row['cantidad_peliculas'] = movies.filter(mask).shape[0]
    data.append(row)
    
cantidad_filmaciones_mes = pl.DataFrame(data=data)
cantidad_filmaciones_mes

month,cantidad_peliculas
str,i64
"""JULIO""",2612
"""ENERO""",5858
"""OCTUBRE""",4563
"""AGOSTO""",3358
"""MARZO""",3513
…,…
"""JUNIO""",3106
"""ABRIL""",3423
"""FEBRERO""",3006


Esta tablita de 12 filas y 2 columnas responde la necesidad de la Primera funcion de la Api

In [65]:
cantidad_filmaciones_mes.write_parquet("data/data_api/api_01_cantidad_filmaciones_mes.parquet")

***
## **Trabajando en `cantidad_filmaciones_dia`**

**REQUERIMIENTO DE LA FUNCION**<br>
Se ingresa un `día en idioma Español`. Debe devolver la `cantidad de películas` que fueron estrenadas en `día consultado` en la totalidad del dataset.

In [23]:
days = movies['name_of_day'].unique().to_list()

# Cada fila se mete (en cada iteracion) en un diccionario llamado `row` y estos a su vez en una lista llamada `data`
data = []
for day in days:
    row = {}
    mask = movies['name_of_day'] == day
    row['day'] = day
    row['cantidad_peliculas'] = movies.filter(mask).shape[0]
    data.append(row)

cantidad_filmaciones_dia = pl.DataFrame(data=data)
cantidad_filmaciones_dia

day,cantidad_peliculas
str,i64
"""LUNES""",3483
"""JUEVES""",7460
"""SABADO""",5096
"""MIERCOLES""",6955
"""VIERNES""",13786
"""MARTES""",4578
"""DOMINGO""",3571


Esta tablita de 7 x 2 responde las necesidades de la Segunda funcion de la Api

In [67]:
cantidad_filmaciones_dia.write_parquet('data/data_api/api_02_cantidad_filmaciones_dia.parquet')

***
## **Trabajando en `score_titulo` y `votos_titulo`**

**REQUERIMIENTO DE LAS FUNCIONES**<br>
- `score_titulo`:
    - Se ingresa el título de una filmación esperando como respuesta el título, el año de estreno y el score
- `votos_titulo`:
    - Se ingresa el `título` de una filmación esperando como respuesta el `título`, la `cantidad de votos` y el `valor promedio de las votaciones`. La misma variable deberá de contar con al menos `2000 valoraciones`, caso contrario, debemos contar con un mensaje avisando que no cumple esta condición y que por ende, no se devuelve ningun valor.

In [24]:
score_votos_titulo = movies[['title', 'vote_count', 'vote_average', 'release_year', 'popularity']]
# Pasaré a Mayusculas y eliminaré posibles espacios en blanco al principio o final del titulo
score_votos_titulo = score_votos_titulo.with_columns(
    pl.col('title').str.to_uppercase().str.strip_chars().alias('title')
)
score_votos_titulo.head()

title,vote_count,vote_average,release_year,popularity
str,i32,f32,i32,f32
"""TOY STORY""",5415,7.7,1995,21.946943
"""JUMANJI""",2413,6.9,1995,17.015539
"""GRUMPIER OLD MEN""",92,6.5,1995,11.7129
"""WAITING TO EXHALE""",34,6.1,1995,3.859495
"""FATHER OF THE BRIDE PART II""",173,5.7,1995,8.387519


### **Como sería la Lógica para obtener la respuesta requerida para `score_titulo`**


In [25]:
"""
# Caso 1:
#  toy StOrY --> muestra la informacion requerida de la pelicula
# Caso 2:
#  TOYE STORY --> está mal escrita arroja error de busqueda
"""

titulos_filmaciones = ['toy StOrY', 'TOYE STORY']

for titulo_filmacion in titulos_filmaciones:
    mask = score_votos_titulo["title"] == titulo_filmacion.strip().upper()
    try:
        popularity = score_votos_titulo.filter(mask)["popularity"][0]
        year = score_votos_titulo.filter(mask)["release_year"][0]
        
        message = f"""La Película {titulo_filmacion} fue estrenada el año {year} cuenta con una popularidad de {popularity}"""
        print(f'titulo --> {titulo_filmacion}')
        print(f'año --> {year}')
        print(f'popularidad/score --> {popularity}')
    except IndexError:
        message = 'status_code=404, detail="INCORRECTO"'
    finally:
        print(message)
        print('*'*50)
        
# NOTA: logicamente en FastApi las respuesta se envian dentro de Clases que heredan de BaseModel de la Libreria
#       pydantic como lo sugiere la documentacion oficial de FastApi. 

titulo --> toy StOrY
año --> 1995
popularidad/score --> 21.946943283081055
La Película toy StOrY fue estrenada el año 1995 cuenta con una popularidad de 21.946943283081055
**************************************************
status_code=404, detail="INCORRECTO"
**************************************************


### **Como sería la Lógica para obtener la respuesta requerida para `votos_titulo`**

In [26]:
"""
# Casos 1:
#  toy StOrY --> muestra la informacion requerida de la pelicula
# Casos 2:
#  TOYE STORY --> está mal escrita arroja error de busqueda
# Casos 3:
#  GRUMPIER OLD MEN --> No alcanza las 2000 valoraciones devuelve el mensaje acordado
"""

titulos_filmaciones = ['toy StOrY', 'TOYE STORY', 'GRUMPIER OLD MEN' ]

for titulo_filmacion in titulos_filmaciones:
    mask = score_votos_titulo["title"] == titulo_filmacion.strip().upper()
    try:
        vote_count = score_votos_titulo.filter(mask)["vote_count"][0]
        vote_average = score_votos_titulo.filter(mask)["vote_average"][0]
        year = score_votos_titulo.filter(mask)["release_year"][0]
        if vote_count >= 2000:
            message = f"""La Película {titulo_filmacion} fue estrenada el año {year} cuenta con un total de {vote_count} valoraciones, con un promedio de {vote_average}"""
            print(f'vote_count --> {vote_count}')
            print(f'vote_average --> {vote_average}')
        else:
            message = f"La Película {titulo_filmacion} no cuenta con 2000 valoraciones por ende no se muestra data"
        print(message)
    except IndexError:
        print('status_code=404, detail="INCORRECTO"')
    finally:
        print('*'*50)
        
# NOTA: logicamente en FastApi las respuesta se envian dentro de Clases que heredan de BaseModel de la Libreria
#       pydantic como lo sugiere la documentacion oficial de FastApi. 
        

vote_count --> 5415
vote_average --> 7.699999809265137
La Película toy StOrY fue estrenada el año 1995 cuenta con un total de 5415 valoraciones, con un promedio de 7.699999809265137
**************************************************
status_code=404, detail="INCORRECTO"
**************************************************
La Película GRUMPIER OLD MEN no cuenta con 2000 valoraciones por ende no se muestra data
**************************************************


In [74]:
score_votos_titulo.write_parquet('data/data_api/api_03_04_titulos.parquet')

***
## **Trabajando en `get_actor`**

**REQUERIMIENTO DE LA FUNCION**<br>
Se ingresa el nombre de un `actor` que se encuentre dentro de un dataset debiendo devolver el `éxito del mismo` medido a través del `retorno`. Además, la `cantidad de películas` que en las que ha participado y el `promedio de retorno`. La definición no deberá considerar directores.<br>


**NOTA:** Para poder cumplir con lo exígido por esta función debo traer data del dataset `credit.csv` procesada para este fin en concreto, en el archivo `actores.parquet`.

In [27]:
actores_df = pl.read_parquet('data/actores.parquet')
# Creo la Columna `total_return` y la casteo a Float64
actores_df = actores_df.with_columns(pl.lit(0).alias('total_return'))
actores_df = actores_df.with_columns(pl.col("total_return").cast(pl.Float64).alias('total_return'))

actores_df = actores_df.with_columns(pl.lit(0).alias('number_movies_found'))

actores_df.head(1)
# Nota: en el Campo `movies` hay una lista con los id de las películas en que a participado cada actor

id_actor,actor,movies,number_of_movies,total_return,number_movies_found
i64,str,list[i64],i64,f64,i32
120646,"""NINO FRASSICA""","[25846, 37710, … 320316]",8,0.0,0


In [28]:
# Obtengo la Lista de actores y lo llevo a la variable actores
actores = actores_df['actor'].to_list()

for actor in actores:
    mask = actores_df['actor'] == actor
    # Obtengo la lista de Peliculas en que a trabajado cada actor
    movies_list = actores_df.filter(mask)['movies'][0].to_list()

    # en la variable list_movies_found Borrare las movies_id que aparecen en credits.csv 
    # pero que no estan en movies_dataset.csv
    list_movies_found = movies_list.copy()

    total_return = 0
    for movie_id in movies_list:
        # filtro mi dataframe `movies` con las peliculas en que a trabaja el actor de la respectiva iteracion
        mask = movies['id'] == movie_id
        
        # Si no aparece en movies se rompe, arroja error, salto la operacion y en except borro ese id de los encontrados
        try:
          total_return += movies.filter(mask)['return'][0]
        except:
          list_movies_found.remove(movie_id)  

    number_movies_found = len(list_movies_found)
    
    actores_df = actores_df.with_columns(
        pl.when(pl.col('actor') == actor)
          .then(pl.lit(total_return))
          .otherwise(pl.col('total_return'))
          .alias('total_return')
    )

    actores_df = actores_df.with_columns(
        pl.when(pl.col('actor') == actor)
          .then(pl.lit(number_movies_found))
          .otherwise(pl.col('number_movies_found'))
          .alias('number_movies_found')
    )

**NOTA**: En el paso anterior sume todos los retornos, y lo guardé en `total_return`.  El numero de peliculas en la que actua (segun credit.csv), no lo tomaré en cuenta para el calculo del average, dado que hay ocaciones donde un actor tiene `N` ids de películas según el Dataset `credit.csv`, pero en el dataset `movies_dataset.csv` existen sólo (`N-1` ó `N-2`..etc) peliculas (son pocos los casos, pero los hay).   Esos `Poquísimos` casos son los que atrapa el `except del Bloque anterior` y utilizo una nueva variable llamada `number_movies_found` para ser mas preciso en el `promedio de retornos` 

Creo el promedio de los retornos

In [29]:
# Obviamente el denominador no puede ser 0
# Cuando `number_movies_found` > 0 entonces `total_return`/`number_movies_found` en otro caso 0
actores_df = actores_df.with_columns(
    pl.when(pl.col('number_movies_found') > 0)
      .then(pl.col('total_return') / pl.col('number_movies_found'))
      .otherwise(pl.lit(0))
      .alias('average_return')
)
actores_df.head(1)

id_actor,actor,movies,number_of_movies,total_return,number_movies_found,average_return
i64,str,list[i64],i64,f64,i32,f64
120646,"""NINO FRASSICA""","[25846, 37710, … 320316]",8,5.287314,8,0.660914


### **Como sería la Lógica para obtener la respuesta requerida**

In [30]:
# En este caso Muestra la información del Primero, pero arrojar error de busqueda en el segundo `Harrisonn` le sobra una `n`
nombres_actores = ['HaRRison fOrd', 'Harrisonn ford']

for nombre_actor in nombres_actores:
    mask = actores_df["actor"] == nombre_actor.strip().upper()
    try:
        # El actor X ha participado de X cantidad de filmaciones, el mismo ha conseguido un 
        # retorno de X con un promedio de X por filmación
        number_movies = actores_df.filter(mask)["number_of_movies"][0]
        total_return = actores_df.filter(mask)["total_return"][0]
        average_return = actores_df.filter(mask)["average_return"][0]
        print(f'actor --> {nombre_actor}')
        print(f'number_movies  --> {number_movies}')
        print(f'total_return --> {total_return}')
        print(f'average_return --> {average_return}')
        message = f'El actor {nombre_actor} ha participado de {number_movies} filmaciones, con un retorno de {total_return} con un promedio de {average_return}'
    except IndexError:
        message = 'HTTPException(status_code=404, detail="INCORRECTO")'

    finally:
        print(message)
        print('*'*50)

# Nota: Obviamente Ajustando Esto a FastApi

actor --> HaRRison fOrd
number_movies  --> 58
total_return --> 434.92924922704697
average_return --> 7.4987801590870165
El actor HaRRison fOrd ha participado de 58 filmaciones, con un retorno de 434.92924922704697 con un promedio de 7.4987801590870165
**************************************************
HTTPException(status_code=404, detail="INCORRECTO")
**************************************************


In [79]:
actores_df.write_parquet('data/data_api/api_05_get_actor.parquet')

***
## **Trabajando en `get_director`**

**REQUERIMIENTO DE LA FUNCION**<br>
Se ingresa el `nombre de un director` que se encuentre dentro de un dataset debiendo devolver el `éxito del mismo medido a través del retorno`. Además, deberá devolver el `nombre de cada película` con la `fecha de lanzamiento`, `retorno individual`, `costo` y `ganancia` de la misma.<br>


**NOTA:** Para poder cumplir con lo exígido por esta función debo traer data del dataset `credit.csv` procesada para este fin en concreto `directores.parquet`.

In [31]:
director = pl.read_parquet('data/directores.parquet')
get_director = movies[['id', 'title', 'release_date', 'revenue', 'budget', 'return']]
get_director.columns = ['id', 'title', 'release_date', 'revenue', 'budget', 'returned']

# Grabo la data de las peliculas solo con lo requerido
get_director.write_parquet('data/data_api/api_06_get_director.parquet')

Para esta Función de la API se hace mas práctivo manejar la información en dos tablas o archivos `parquet`
- api_06_get_director.parquet
- directores.parquet

### **Como sería la Lógica para obtener la respuesta requerida**

In [32]:
# este caso el primero no lo va a encontrar porque esta mal escrito (Stevens) sobra la `s` final y arroja error.
# el segundo si lo localiza y muestra la información
nombres_directores = ['Stevens Spielberg', 'StevEn SpIeLberg']

for nombre_director in nombres_directores:
    try:
        # Obteniendo la lista de id de peliculas de la tabla directores
        mask = director["director"] == nombre_director.strip().upper()
        movies_list = director.filter(mask)["movies"].to_list()[0]

        # Filtrando la tabla get_director y obteniendo la informacion
        mask = get_director["id"].is_in(movies_list)
        requiered_movies = get_director.filter(mask).to_dicts()
        total_return = get_director.filter(mask)["returned"].sum()

        print(f'Nombre del Director --> {nombre_director}')
        print(f'Total Return --> {total_return}')
        print('PELICULAS INDIVIDUALES')
        for i in range(len(requiered_movies)):
            print(requiered_movies[i])

    except IndexError:
        print('HTTPException status_code=404, detail="INCORRECTO"')
    finally:
        print('*'*50)

HTTPException status_code=404, detail="INCORRECTO"
**************************************************
Nombre del Director --> StevEn SpIeLberg
Total Return --> 309.99810791015625
PELICULAS INDIVIDUALES
{'id': 329, 'title': 'Jurassic Park', 'release_date': datetime.date(1993, 6, 11), 'revenue': 920099968.0, 'budget': 63000000.0, 'returned': 14.604761123657227}
{'id': 424, 'title': "Schindler's List", 'release_date': datetime.date(1993, 11, 29), 'revenue': 321365568.0, 'budget': 22000000.0, 'returned': 14.607525825500488}
{'id': 601, 'title': 'E.T. the Extra-Terrestrial', 'release_date': datetime.date(1982, 4, 3), 'revenue': 792965312.0, 'budget': 10500000.0, 'returned': 75.5205078125}
{'id': 85, 'title': 'Raiders of the Lost Ark', 'release_date': datetime.date(1981, 6, 12), 'revenue': 389925984.0, 'budget': 18000000.0, 'returned': 21.662553787231445}
{'id': 89, 'title': 'Indiana Jones and the Last Crusade', 'release_date': datetime.date(1989, 5, 24), 'revenue': 474171808.0, 'budget': 48

Grabo la data movie en un archivo para poder utilizarlo en el procesamiento para Normalizar y Modelo

In [31]:
movies.write_parquet('data/movies.parquet')